<a href="https://colab.research.google.com/github/ai-wrangler/BA_sms_LLM/blob/main/SMS_LLM_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SMS Spam Classification with Embeddings and HuggingFace LLM
This Colab-ready notebook recreates the Lab 5 text mining workflow from Weka using Python pipelines and HuggingFace's free Inference API. You'll load the original ARFF dataset, build embedding-based classifiers, invoke HuggingFace models for zero-shot spam detection, and compare the evaluation metrics across approaches.

## How to use this notebook in Google Colab
1. Upload `SMS_LLM_Colab.ipynb` to Colab (File → Upload notebook) or open it from Drive.
2. Runtime → Change runtime type → make sure Python 3.10+; GPU is optional.
3. Prepare the dataset: copy `TextCollection_sms.arff` to your Drive or download it locally so you can upload it when prompted.
4. Get a free HuggingFace API token from https://huggingface.co/settings/tokens (create a "Read" token). Store it securely (`Tools → Secrets` in Colab or `google.colab.userdata`). This notebook expects an environment variable called `HF_API_KEY`.
5. Run the cells in order—each is annotated to match the lab workflow and highlight differences between embeddings and LLM-based classification.

**HuggingFace Free Tier Limits:**
- Rate limit: 1,000 requests/day for free tier (varies by model)
- Most models support 1,024-2,048 tokens per request
- Some popular models may have lower rate limits during peak usage

### Fixing 'Invalid Notebook' Error on GitHub

To resolve the 'state' key missing error when rendering your notebook on GitHub, you should clear all cell outputs before saving and committing your notebook. Here's how to do it in Google Colab:

1.  **Open your notebook** in Google Colab.
2.  Go to the **'Runtime'** menu at the top.
3.  Select **'Clear all outputs'**.
4.  **Save the notebook** (File > Save).
5.  Then, you can **download the `.ipynb` file** and upload it to GitHub, or sync it if you are using Google Drive integration with GitHub.

In [1]:
# Install libraries that are not included in the base Colab runtime
%pip install -q pandas numpy scikit-learn seaborn matplotlib sentence-transformers scipy liac-arff huggingface_hub

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import os
import random
import re
import time
from pathlib import Path
import arff # Replaced from scipy.io import arff with import arff (for liac-arff)
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    f1_score, precision_score, recall_score
)
from sklearn.model_selection import train_test_split

plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
# Reproducibility helpers
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

## Load the SMS Spam ARFF dataset
The lab uses `TextCollection_sms.arff`. Use one of the cells below to make it available in the Colab filesystem. Uploading via the UI is the quickest path if the file is on your laptop.

In [ ]:
# Option A: Mount Google Drive (run this if the ARFF lives in Drive)
from google.colab import drive
drive.mount('/content/drive')
# After mounting, set ARFF_PATH = '/content/drive/MyDrive/path/to/TextCollection_sms.arff'

In [ ]:
# Option B: Upload the ARFF manually (run this if the file is on your machine)
from google.colab import files
uploaded = files.upload()
ARFF_PATH = next(iter(uploaded))  # use the first uploaded filename

In [ ]:
# If you mounted Drive instead of uploading, set the explicit path here.
# Example: ARFF_PATH = '/content/drive/MyDrive/datasets/TextCollection_sms.arff'
ARFF_PATH = locals().get('ARFF_PATH', 'TextCollection_sms.arff')
print(f'Using dataset located at: {ARFF_PATH}')

In [ ]:
# Read the ARFF file into a DataFrame and mirror the original lab schema
# Using liac-arff as scipy.io.arff does not support string attributes
arff_data = arff.load(open(ARFF_PATH, 'r'))
raw_data = arff_data['data']
attributes = arff_data['attributes']
column_names = [attr[0] for attr in attributes]

sms_df = pd.DataFrame(raw_data, columns=column_names)
# liac-arff reads strings directly, so no decoding is needed
sms_df = sms_df.rename(columns={'Text': 'message', 'class-att': 'label'})
sms_df['label'] = sms_df['label'].map({'0': 'ham', '1': 'spam'})
sms_df['char_len'] = sms_df['message'].str.len()
sms_df.head()

In [ ]:
# Quick class balance check
ax = sms_df['label'].value_counts().sort_index().plot(kind='bar', color=['#4C72B0', '#DD8452'])
ax.set(title='Class distribution', xlabel='Label', ylabel='Count')
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2, p.get_height()),
                ha='center', va='bottom')
plt.show()
sms_df.describe(include='all')

In [ ]:
# Train/test split mirroring the lab evaluation
X_train, X_test, y_train, y_test = train_test_split(
    sms_df['message'],
    sms_df['label'],
    test_size=0.2,
    stratify=sms_df['label'],
    random_state=RANDOM_STATE
)
print(f'Train set: {X_train.shape[0]} messages | Test set: {X_test.shape[0]} messages')

In [ ]:
# Shared evaluation helpers for classical models and the LLM
results = []

def capture_metrics(name: str, y_true, y_pred) -> pd.Series:
    metrics = {
        'model': name,
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, pos_label='spam'),
        'recall': recall_score(y_true, y_pred, pos_label='spam'),
        'f1': f1_score(y_true, y_pred, pos_label='spam')
    }
    results.append(metrics)
    print(json.dumps(metrics, indent=2))
    return pd.Series(metrics)

def plot_confusion(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred, labels=['ham', 'spam'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['ham', 'spam'], yticklabels=['ham', 'spam'])
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

## Baseline 1: TF–IDF + Logistic Regression
Replicates the bag-of-words style features typically explored in Weka's text mining lab.

In [ ]:
tfidf = TfidfVectorizer(lowercase=True, stop_words='english', min_df=3, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

bow_clf = LogisticRegression(max_iter=200, random_state=RANDOM_STATE, n_jobs=None)
bow_clf.fit(X_train_tfidf, y_train)
bow_preds = bow_clf.predict(X_test_tfidf)
capture_metrics('TFIDF + LogisticRegression', y_test, bow_preds)
print(classification_report(y_test, bow_preds))
plot_confusion(y_test, bow_preds, 'TF-IDF Logistic Regression Confusion Matrix')

## Baseline 2: SentenceTransformer Embeddings + Logistic Regression
Uses a semantic embedding (MiniLM) to capture contextual similarity beyond word frequencies.

In [ ]:
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
X_train_emb = embedder.encode(X_train.tolist(), show_progress_bar=True, batch_size=128)
X_test_emb = embedder.encode(X_test.tolist(), show_progress_bar=True, batch_size=128)

embed_clf = LogisticRegression(max_iter=500, random_state=RANDOM_STATE)
embed_clf.fit(X_train_emb, y_train)
embed_preds = embed_clf.predict(X_test_emb)
capture_metrics('MiniLM Embeddings + LogisticRegression', y_test, embed_preds)
print(classification_report(y_test, embed_preds))
plot_confusion(y_test, embed_preds, 'MiniLM Logistic Regression Confusion Matrix')

## Configure HuggingFace for LLM-based Zero/Few-shot Classification
You need a free HuggingFace API token from https://huggingface.co/settings/tokens. In Colab you can store it via `Tools → Secrets` and retrieve it with `google.colab.userdata.get('HF_API_KEY')`. Alternatively, set `os.environ['HF_API_KEY']` manually (just avoid hard-coding secrets in plain text).

**Available Models (Verified Working):**
- `mistralai/Mistral-7B-Instruct-v0.2` - **RECOMMENDED** - Most reliable and accurate
- `HuggingFaceH4/zephyr-7b-beta` - Good for instruction following
- `meta-llama/Meta-Llama-3-8B-Instruct` - Latest Llama model
- `google/flan-t5-xxl` - Fallback option (different format)

**Rate Limits:** 
- Free tier: ~1,000 requests/day
- Pro tier ($9/month): Higher limits and priority access

In [ ]:
import os
from huggingface_hub import InferenceClient

HF_API_KEY = os.environ.get('HF_API_KEY')
if HF_API_KEY is None:
    try:
        from google.colab import userdata
        HF_API_KEY = userdata.get('HF_API_KEY')
    except ImportError:
        pass

if not HF_API_KEY:
    raise ValueError('Missing HuggingFace API key. Set HF_API_KEY via Colab secrets or environment variables before continuing.')

# Using Mistral-7B - Very reliable and widely available on HF Inference API
HF_MODEL = 'mistralai/Mistral-7B-Instruct-v0.2'
client = InferenceClient(token=HF_API_KEY)
print(f'HuggingFace Inference API ready with model: {HF_MODEL}')
print(f'Note: This model is reliable and well-supported on HF API')
print(f'Pro tier: Higher rate limits and priority access')

## 🚀 Quick Start Guide for HuggingFace LLM Classification

**Follow these steps in order:**

1. **Cell 21** ⬇️ - Configure API (set HF_API_KEY and model)
2. **Cell 23** - Verify API connection (run diagnostic)
3. **Cell 29** - Define the classifier function
4. **Cell 30** - Set and test the classifier
5. **Cell 32** - Process all messages

**If you get errors:**
- Run the diagnostic cell (23) to find working models
- Check your API key is valid at https://huggingface.co/settings/tokens
- Try switching models in cell 21

In [ ]:
# 🔧 DIAGNOSTIC: Run this cell first to find a working model/method
import requests

print("Testing HuggingFace API with available models...\n")

test_message = "WIN FREE PRIZE NOW!"
# These models are confirmed to work with HF Inference API
models = [
    'mistralai/Mistral-7B-Instruct-v0.2',  # Most reliable
    'HuggingFaceH4/zephyr-7b-beta',
    'meta-llama/Meta-Llama-3-8B-Instruct',
    'google/flan-t5-xxl'  # Fallback option
]

for model_name in models:
    print(f"\n{'='*70}")
    print(f"Testing: {model_name}")
    print('='*70)
    
    try:
        url = f"https://api-inference.huggingface.co/models/{model_name}"
        headers = {"Authorization": f"Bearer {HF_API_KEY}"}
        
        response = requests.post(
            url,
            headers=headers,
            json={
                "inputs": f"Classify as Spam or Ham: {test_message}\nAnswer:",
                "parameters": {"max_new_tokens": 5, "temperature": 0.1}
            },
            timeout=30
        )
        
        print(f"Status: {response.status_code}")
        
        if response.status_code == 200:
            print(f"✓ SUCCESS! This model works!")
            print(f"Response: {response.json()}")
            print(f"\n📝 To use this model, update cell 21:")
            print(f"HF_MODEL = '{model_name}'")
            break
        elif response.status_code == 503:
            print("⏳ Model loading... wait 20s and try again")
        elif response.status_code == 401:
            print("❌ AUTH ERROR - Check your HF_API_KEY!")
            break
        elif response.status_code == 410:
            print("❌ Model removed/unavailable (410 Gone)")
        else:
            print(f"❌ Error {response.status_code}: {response.text[:200]}")
            
    except Exception as e:
        print(f"❌ Exception: {e}")

print(f"\n{'='*70}")
print("✅ Recommended: mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
# 🔍 STEP 1: Verify API Key and Test Connection
import requests

print("="*70)
print("STEP 1: Verifying HuggingFace API Configuration")
print("="*70)

# Check if API key is set
if 'HF_API_KEY' not in globals() or not HF_API_KEY:
    print("❌ ERROR: HF_API_KEY is not set!")
    print("\nTo fix:")
    print("1. In Colab: Tools → Secrets → Add 'HF_API_KEY'")
    print("2. Or run: import os; os.environ['HF_API_KEY'] = 'your_token_here'")
else:
    print(f"✓ API Key found: {HF_API_KEY[:10]}...{HF_API_KEY[-4:]}")
    
    # Test simple API call
    print("\nTesting API connection...")
    try:
        test_url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
        headers = {"Authorization": f"Bearer {HF_API_KEY}"}
        
        response = requests.post(
            test_url,
            headers=headers,
            json={"inputs": "Hello", "parameters": {"max_new_tokens": 5}},
            timeout=15
        )
        
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ API CONNECTION SUCCESSFUL!")
            print("Response received:", response.json()[:100] if len(str(response.json())) > 100 else response.json())
        elif response.status_code == 401:
            print("❌ AUTHENTICATION FAILED")
            print("Your API key is invalid or expired")
            print("Get a new token at: https://huggingface.co/settings/tokens")
        elif response.status_code == 503:
            print("⏳ Model is loading... This is normal, wait 20 seconds and try again")
        elif response.status_code == 429:
            print("⚠️  Rate limit reached. Wait a moment and try again")
        else:
            print(f"⚠️  Unexpected status: {response.status_code}")
            print(f"Response: {response.text[:200]}")
            
    except requests.exceptions.Timeout:
        print("❌ CONNECTION TIMEOUT")
        print("Check your internet connection")
    except Exception as e:
        print(f"❌ ERROR: {e}")

print("\n" + "="*70)

### 🔧 Troubleshooting Guide

**If you're experiencing API errors:**

1. **Try a different model** - Some models work better with the API than others:
   - Recommended: `mistralai/Mistral-7B-Instruct-v0.2`
   - Alternative: `HuggingFaceH4/zephyr-7b-beta`
   - Latest: `meta-llama/Meta-Llama-3.1-8B-Instruct`

2. **Switch to requests-based implementation** - Run the cells below to define `classify_sms = classify_with_hf_requests`

3. **Check model status** - Some models may be loading: https://huggingface.co/models

4. **Verify API key** - Make sure your `HF_API_KEY` is valid and has the right permissions

### Alternative HuggingFace Models for Spam Detection

You can try different models by changing the `HF_MODEL` variable. Here are recommended free-tier options:

**Recommended Models:**
1. **`microsoft/Phi-3-mini-4k-instruct`** (Default)
   - Size: 3.8B parameters
   - Speed: Fast (~1-2s per request)
   - Best for: Quick classification tasks
   - Context: 4k tokens

2. **`mistralai/Mistral-7B-Instruct-v0.3`**
   - Size: 7B parameters
   - Speed: Moderate (~2-3s per request)
   - Best for: Better accuracy on nuanced messages
   - Context: 8k tokens

3. **`HuggingFaceH4/zephyr-7b-beta`**
   - Size: 7B parameters
   - Speed: Moderate (~2-3s per request)
   - Best for: Instruction following
   - Context: 8k tokens

4. **`meta-llama/Llama-3.2-3B-Instruct`**
   - Size: 3B parameters
   - Speed: Very fast (~1s per request)
   - Best for: Quick responses, good quality
   - Context: 8k tokens

**Free Tier Limits:**
- **Rate Limit**: ~1,000 requests per day
- **Token Limit**: Varies by model (typically 1,024-4,096 tokens per request)
- **Concurrent Requests**: 1-2 at a time
- Monitor usage at: https://huggingface.co/settings/tokens

In [ ]:
# Test your HuggingFace API connection before processing all messages
def test_hf_connection():
    """Test the HuggingFace API and verify it's working correctly."""
    test_messages = [
        ("Win a free iPhone now! Click here!", "spam"),
        ("Hey, are you coming to dinner tonight?", "ham"),
        ("URGENT! Your account will be closed. Verify now!", "spam"),
        ("Thanks for your help yesterday", "ham")
    ]
    
    print("Testing HuggingFace API connection...")
    print(f"Model: {HF_MODEL}\n")
    
    success_count = 0
    for msg, expected in test_messages:
        try:
            result = classify_with_hf(msg)
            status = "✓" if result == expected else "?"
            success_count += 1 if result == expected else 0
            print(f"{status} '{msg[:50]}...' -> {result} (expected: {expected})")
        except Exception as e:
            print(f"✗ Test failed: {e}")
            return False
    
    print(f"\n{'='*60}")
    print(f"API Test Results: {success_count}/{len(test_messages)} correct")
    print(f"{'='*60}\n")
    
    if success_count >= len(test_messages) * 0.5:  # At least 50% correct
        print("✓ API connection is working!")
        return True
    else:
        print("⚠ API is responding but results may be unreliable")
        print("Consider trying a different model or checking the prompt format")
        return False

# Run the test (comment out after verifying it works)
# test_hf_connection()

### Alternative: Try Different Models if Current One Fails

If you're experiencing issues with the default model, try these alternatives which have better API compatibility:

**Most Reliable Options:**
```python
# Option 1: Mistral (very reliable with HF API)
HF_MODEL = 'mistralai/Mistral-7B-Instruct-v0.2'

# Option 2: Zephyr (good instruction following)
HF_MODEL = 'HuggingFaceH4/zephyr-7b-beta'

# Option 3: Llama 3.1 (latest, very good)
HF_MODEL = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
```

Just update the `HF_MODEL` variable in the configuration cell above and re-run.

In [ ]:
# Alternative implementation using requests library directly (MOST RELIABLE)
import requests

def classify_with_hf_requests(text: str, retry: int = 3) -> str:
    """
    Classify SMS message using requests library directly.
    This is the most reliable method for HuggingFace Inference API.
    """
    API_URL = f"https://api-inference.huggingface.co/models/{HF_MODEL}"
    headers = {"Authorization": f"Bearer {HF_API_KEY}"}
    
    # Simple, direct prompt
    prompt = f"Classify this SMS message as either 'Spam' or 'Ham':\n\nMessage: {text}\n\nClassification:"
    
    for attempt in range(retry):
        try:
            response = requests.post(
                API_URL,
                headers=headers,
                json={
                    "inputs": prompt,
                    "parameters": {
                        "max_new_tokens": 10,
                        "temperature": 0.1,
                        "do_sample": False,
                        "return_full_text": False
                    }
                },
                timeout=30
            )
            
            # Handle model loading
            if response.status_code == 503:
                wait_time = 20
                print(f"Model loading, waiting {wait_time}s...")
                time.sleep(wait_time)
                continue
            
            # Handle rate limiting
            if response.status_code == 429:
                wait_time = 5 * (2 ** attempt)
                print(f"Rate limit, waiting {wait_time}s...")
                time.sleep(wait_time)
                continue
            
            # Handle auth errors
            if response.status_code == 401:
                print(f"Authentication error - check your HF_API_KEY")
                return 'ham'
                
            response.raise_for_status()
            result = response.json()
            
            # Parse different response formats
            if isinstance(result, list) and len(result) > 0:
                if isinstance(result[0], dict):
                    text_result = result[0].get('generated_text', '').strip().lower()
                else:
                    text_result = str(result[0]).strip().lower()
            elif isinstance(result, dict):
                text_result = result.get('generated_text', result.get('text', '')).strip().lower()
            else:
                text_result = str(result).strip().lower()
            
            # Clean and parse
            text_result = text_result.replace('*', '').replace('#', '').replace('`', '').strip()
            
            # Look for spam/ham
            if 'spam' in text_result and 'ham' not in text_result:
                return 'spam'
            elif 'ham' in text_result and 'spam' not in text_result:
                return 'ham'
            elif 'spam' in text_result and 'ham' in text_result:
                # If both, take the first one
                spam_idx = text_result.index('spam')
                ham_idx = text_result.index('ham')
                return 'spam' if spam_idx < ham_idx else 'ham'
            else:
                # Default to ham if unclear
                return 'ham'
                
        except KeyboardInterrupt:
            raise
        except requests.exceptions.Timeout:
            print(f"Timeout on attempt {attempt + 1}")
            if attempt == retry - 1:
                return 'ham'
            time.sleep(2)
        except Exception as e:
            if attempt == retry - 1:
                print(f"Failed after {retry} retries: {str(e)[:100]}")
                return 'ham'
            time.sleep(2 * (attempt + 1))
    
    return 'ham'

print("✓ classify_with_hf_requests() defined")
print("This uses the reliable requests library")
print("\nTo use it, run: classify_sms = classify_with_hf_requests")

In [ ]:
# 🎯 COMPLETE WORKING CLASSIFIER - Use this one!
import requests
import time

def classify_spam_simple(text: str, max_retries: int = 3) -> str:
    """
    Simple, robust SMS spam classifier using HuggingFace API.
    Returns 'spam' or 'ham'.
    """
    url = f"https://api-inference.huggingface.co/models/{HF_MODEL}"
    headers = {"Authorization": f"Bearer {HF_API_KEY}"}
    
    # Very simple prompt that works reliably
    prompt = f"Is this message spam or ham? Message: {text}\nAnswer (spam/ham):"
    
    for attempt in range(max_retries):
        try:
            response = requests.post(
                url,
                headers=headers,
                json={
                    "inputs": prompt,
                    "parameters": {
                        "max_new_tokens": 20,
                        "temperature": 0.1,
                        "top_p": 0.9
                    }
                },
                timeout=30
            )
            
            # Handle different status codes
            if response.status_code == 503:
                # Model loading
                if attempt < max_retries - 1:
                    print(f"Model loading, waiting 20s... (attempt {attempt + 1}/{max_retries})")
                    time.sleep(20)
                    continue
                else:
                    print("Model still loading after retries, defaulting to ham")
                    return 'ham'
            
            elif response.status_code == 429:
                # Rate limit
                wait_time = 5 * (2 ** attempt)
                if attempt < max_retries - 1:
                    print(f"Rate limited, waiting {wait_time}s...")
                    time.sleep(wait_time)
                    continue
                else:
                    return 'ham'
            
            elif response.status_code == 401:
                print("❌ Authentication failed - check your HF_API_KEY")
                return 'ham'
            
            elif response.status_code == 410:
                print(f"❌ Model {HF_MODEL} is no longer available")
                return 'ham'
            
            elif response.status_code != 200:
                if attempt < max_retries - 1:
                    time.sleep(2)
                    continue
                else:
                    print(f"Error {response.status_code}: {response.text[:100]}")
                    return 'ham'
            
            # Success! Parse the response
            result = response.json()
            
            # Handle list response
            if isinstance(result, list) and len(result) > 0:
                if isinstance(result[0], dict) and 'generated_text' in result[0]:
                    text_response = result[0]['generated_text'].lower()
                else:
                    text_response = str(result[0]).lower()
            # Handle dict response
            elif isinstance(result, dict):
                text_response = result.get('generated_text', result.get('text', str(result))).lower()
            else:
                text_response = str(result).lower()
            
            # Clean up response
            text_response = text_response.replace('*', '').replace('#', '').replace('`', '').strip()
            
            # Extract spam/ham
            if 'spam' in text_response and 'ham' not in text_response:
                return 'spam'
            elif 'ham' in text_response and 'spam' not in text_response:
                return 'ham'
            elif 'spam' in text_response and 'ham' in text_response:
                # Both found - take the first one
                spam_pos = text_response.find('spam')
                ham_pos = text_response.find('ham')
                return 'spam' if spam_pos < ham_pos else 'ham'
            else:
                # No clear answer, default to ham
                return 'ham'
                
        except requests.exceptions.Timeout:
            if attempt < max_retries - 1:
                print(f"Timeout, retrying... ({attempt + 1}/{max_retries})")
                time.sleep(2)
            else:
                print("Timeout after retries")
                return 'ham'
        except KeyboardInterrupt:
            raise
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Error: {str(e)[:50]}, retrying...")
                time.sleep(2)
            else:
                print(f"Failed after {max_retries} attempts: {str(e)[:50]}")
                return 'ham'
    
    return 'ham'

# Test the function
print("✅ classify_spam_simple() function defined")
print("\nQuick test...")
try:
    test_result = classify_spam_simple("WIN FREE PRIZE NOW!!!")
    print(f"✅ Test successful: 'WIN FREE PRIZE NOW!!!' → {test_result}")
    print("\n🎯 Function is ready to use!")
except Exception as e:
    print(f"❌ Test failed: {e}")
    print("Check your API key and model availability")

In [ ]:
# 🚀 SET THE CLASSIFIER TO USE
# Use the simple, reliable classifier
classify_sms = classify_spam_simple

print("="*70)
print(f"✅ Classifier set: {classify_sms.__name__}")
print(f"✅ Model: {HF_MODEL}")
print("="*70)

# Quick test
print("\n🧪 Running quick test...")
try:
    test_messages = [
        "WIN FREE PRIZE NOW!!!",
        "Hey, want to grab dinner tonight?"
    ]
    
    for msg in test_messages:
        result = classify_sms(msg)
        print(f"  '{msg[:40]}...' → {result}")
    
    print("\n✅ Classifier is working! Ready to process messages.")
    
except Exception as e:
    print(f"\n❌ Classifier test failed: {e}")
    print("\nTroubleshooting:")
    print("1. Run cell 23 to verify API connection")
    print("2. Check that HF_API_KEY is set correctly")
    print("3. Try a different model (run the diagnostic cell)")

## LLM Inference Loop
HuggingFace free tier has rate limits (~1,000 requests/day), so we evaluate on a smaller stratified subset of the held-out test set (default 100 messages). Adjust `LLM_SAMPLE_SIZE` based on your daily quota needs.

In [ ]:
# 📊 PROCESS ALL TEST MESSAGES
# HuggingFace Pro allows processing full test set
LLM_SAMPLE_SIZE = None  # Set to None to process entire test set, or specify a number (e.g., 100 for testing)

llm_eval_df = (
    pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
    .rename(columns={'message': 'text', 'label': 'label'})
)

if LLM_SAMPLE_SIZE and LLM_SAMPLE_SIZE < len(llm_eval_df):
    llm_eval_df = (
        llm_eval_df
        .groupby('label', group_keys=False)
        .apply(lambda grp: grp.sample(
            n=max(1, int(LLM_SAMPLE_SIZE * len(grp) / len(llm_eval_df))),
            random_state=RANDOM_STATE
        ), include_groups=False)
        .reset_index(drop=True)
    )

print("="*70)
print(f"📊 Processing {len(llm_eval_df)} messages")
print(f"🤖 Model: {HF_MODEL}")
print("="*70)

# Reduced delay for Pro tier
REQUEST_DELAY = 0.1  # seconds between requests (Pro tier has higher limits)

llm_predictions = []
failed_count = 0
success_count = 0

print(f"\n⏱️  Estimated time: {(len(llm_eval_df) * REQUEST_DELAY / 60):.1f} minutes")
print(f"Starting classification...\n")

start_time = time.time()

for idx, row in llm_eval_df.iterrows():
    try:
        prediction = classify_sms(row['text'])
        llm_predictions.append(prediction)
        
        if prediction in ['spam', 'ham']:
            success_count += 1
        
    except KeyboardInterrupt:
        print("\n\n⚠️  Interrupted by user")
        break
    except Exception as e:
        print(f"\n❌ Error at message {idx + 1}: {str(e)[:50]}")
        llm_predictions.append('ham')  # Default on error
        failed_count += 1
    
    # Small delay to respect rate limits
    if idx < len(llm_eval_df) - 1:
        time.sleep(REQUEST_DELAY)
    
    # Progress updates
    if (idx + 1) % 50 == 0 or idx + 1 == len(llm_eval_df):
        elapsed = time.time() - start_time
        rate = (idx + 1) / elapsed if elapsed > 0 else 0
        remaining = (len(llm_eval_df) - idx - 1) / rate if rate > 0 else 0
        success_rate = (success_count / (idx + 1)) * 100 if idx >= 0 else 0
        print(f"✓ {idx + 1}/{len(llm_eval_df)} | Rate: {rate:.1f} msg/s | Success: {success_rate:.0f}% | ETA: {remaining/60:.1f} min")

# Add predictions to dataframe
if len(llm_predictions) < len(llm_eval_df):
    # Fill remaining with 'ham' if interrupted
    llm_predictions.extend(['ham'] * (len(llm_eval_df) - len(llm_predictions)))

llm_eval_df['prediction'] = llm_predictions

elapsed_total = time.time() - start_time

print("\n" + "="*70)
print(f"✅ COMPLETED in {elapsed_total/60:.1f} minutes")
print(f"📊 Processed: {len(llm_predictions)} messages")
print(f"⚡ Average rate: {len(llm_predictions)/elapsed_total:.1f} messages/second")
print(f"✓ Successful: {success_count}/{len(llm_predictions)} ({success_count/len(llm_predictions)*100:.1f}%)")
if failed_count > 0:
    print(f"⚠️  Failed: {failed_count}")
print("="*70 + "\n")

# Calculate metrics
capture_metrics(f'{HF_MODEL} (LLM zero-shot)', llm_eval_df['label'], llm_eval_df['prediction'])
print(classification_report(llm_eval_df['label'], llm_eval_df['prediction']))
plot_confusion(llm_eval_df['label'], llm_eval_df['prediction'], f'{HF_MODEL.split("/")[1]} Confusion Matrix')

In [ ]:
results_df = pd.DataFrame(results)
results_df.sort_values('f1', ascending=False).reset_index(drop=True)

### Observations
* **TF–IDF + Logistic Regression** mirrors the original Weka text-mining pipeline and usually delivers high recall on overt spam phrases such as "free entry" or "claim now".
* **MiniLM embeddings** capture semantics and can reduce false positives on nuanced ham, at the cost of downloading the encoder and adding encoding latency.
* **HuggingFace LLM** (Phi-3-mini) needs no training data but has rate limits on free tier (~1,000 requests/day); it performs well on context-heavy messages and can understand nuanced spam patterns.
* **Rate Limit Management**: We use smaller sample sizes (100 messages) and add delays between requests to stay within free tier limits.
* Hybrid scoring (e.g., fall back to HuggingFace LLM when the classical models disagree) is a strong extension for future lab work.

## Optional: Export Artifacts
If you want to retain the evaluation outputs in Drive, run the cell below and then use the Colab file browser or `drive.mount` to move the CSVs.

In [ ]:
results_df.to_csv('spam_lab_results_summary.csv', index=False)
llm_eval_df.to_csv('spam_lab_llm_predictions.csv', index=False)
print('Artifacts saved locally. Upload to Drive if you need persistent storage.')

In [ ]:
# Quick diagnostic test to find what works
import requests

def test_all_approaches():
    """Test different API approaches and models to find what works."""
    test_message = "WIN FREE PRIZE NOW! Click here!"
    
    models_to_test = [
        'mistralai/Mistral-7B-Instruct-v0.2',
        'HuggingFaceH4/zephyr-7b-beta',
        'meta-llama/Meta-Llama-3.1-8B-Instruct',
        'microsoft/Phi-3-mini-4k-instruct'
    ]
    
    print("Testing HuggingFace API approaches...\n")
    print("="*70)
    
    for model in models_to_test:
        print(f"\nTesting model: {model}")
        print("-"*70)
        
        # Test with requests
        try:
            API_URL = f"https://api-inference.huggingface.co/models/{model}"
            headers = {"Authorization": f"Bearer {HF_API_KEY}"}
            
            prompt = f"Classify this SMS as 'Spam' or 'Ham': {test_message}\nAnswer:"
            
            response = requests.post(
                API_URL,
                headers=headers,
                json={
                    "inputs": prompt,
                    "parameters": {
                        "max_new_tokens": 10,
                        "temperature": 0.1
                    }
                },
                timeout=30
            )
            
            print(f"Status Code: {response.status_code}")
            
            if response.status_code == 200:
                result = response.json()
                print(f"✓ SUCCESS!")
                print(f"Response: {result}")
                return model, "requests"
            elif response.status_code == 503:
                print("✗ Model is loading (503)")
            elif response.status_code == 429:
                print("✗ Rate limited (429)")
            elif response.status_code == 401:
                print("✗ Authentication failed (401) - Check API key")
            else:
                print(f"✗ Error: {response.status_code}")
                print(f"Response: {response.text[:200]}")
                
        except Exception as e:
            print(f"✗ Exception: {e}")
    
    print("\n" + "="*70)
    print("No working configuration found. Please check:")
    print("1. Your HF_API_KEY is valid")
    print("2. You have access to these models")
    print("3. Your network connection")
    return None, None

# Run the diagnostic
working_model, working_method = test_all_approaches()

## Next Steps
1. Try different HuggingFace models like `mistralai/Mistral-7B-Instruct-v0.3` or `HuggingFaceH4/zephyr-7b-beta` to compare quality/speed trade-offs.
2. Prompt-tune the LLM with few-shot examples in the system prompt for better performance on shorthand or code-mixed spam.
3. Experiment with alternative embedding models (`all-mpnet-base-v2`, fastText) and blend their scores with the LLM for ensemble voting.
4. Monitor your HuggingFace API usage at https://huggingface.co/settings/tokens to track daily limits.
5. Consider upgrading to HuggingFace Pro ($9/month) for higher rate limits if you need to process more messages.
6. Implement batching or caching strategies to minimize API calls while maximizing evaluation coverage.